In [849]:
import binance_executor as bex
import importlib
importlib.reload(bex);

### Top of the orderbook prices and limit order price determination

In [783]:
best_bid, best_ask = bex.get_bba_price(market_type='SPOT', symbol='USDCUSDT')
best_bid, best_ask

(0.9998, 0.9999)

In [785]:
price, best_bid, best_ask = bex.get_mid_limit_price(market_type='SPOT', symbol='ETHUSDT', side='BUY')
print(f"""
best_bid: {best_bid} 
mid-point: {(best_bid + best_ask) / 2}
best_ask: {best_ask}

final limit ordre price: {price}
- mid-point price, decimal precision rounded with reference to quoted price
- ensured rounded price do not cross the book 
""")

Auto-calculated limit price at mid (if-available): 2441.7

best_bid: 2441.7 
mid-point: 2441.705
best_ask: 2441.71

final limit ordre price: 2441.7
- mid-point price, decimal precision rounded with reference to quoted price
- ensured rounded price do not cross the book 



In [816]:
# Balances
spot_balanaces = pd.DataFrame(bex.get_spot_balances())
futures_balances = pd.DataFrame(bex.get_futures_balances())

spot_balanaces[spot_balanaces.asset == 'USDC']

,asset,free,locked
4,USDC,200.08369300,0.00000000


### Executor command functions

In [826]:
# LIMIT prevented from crossing orderbook (Post-only)
bex.submit_spot_order(
    symbol='USDCUSDT', 
    side='SELL',
    quantity=10,
    order_type='LIMIT',
    price=0.98,
    time_in_force='GTC',
    cancel_existing=True
)

Reject Spot SELL Limit: price 0.98 crosses best bid 0.9998


In [832]:
# LIMIT prevented from crossing orderbook (Post-only)
bex.submit_fut_order(
    symbol='ETHUSDT', 
    side='BUY',
    quantity=0.01, # perp contract size quantity (1:1)
    order_type='LIMIT',
    price=2500,
    time_in_force='GTC',
    cancel_existing=True,
    position_side='LONG' # hedge-mode
)

Reject Futures BUY Limit: price 2500 crosses best ask 2441.44


In [824]:
# cancel spot order for a symbol
bex.cancel_existing_orders('USDCUSDT', 'SPOT')

Cancelled existing SPOT limit order 999863829 for USDCUSDT


In [830]:
# cancel futures order for a symbol
bex.cancel_existing_orders('ETHUSDT', 'FUT')

Cancelled existing FUT limit order 8389765908251359792 for ETHUSDT


In [836]:
# close all open futures position for a symbol
bex.close_all_positions(symbol='ETHUSDT')

### Executor request to target based on current balances

In [842]:
order_request = {
    'symbol': 'ETHUSDT', 
    'target_quantity': None,
    'target_notional': 200, # specifying notional target
    'market_type': 'SPOT', 
    'tol_pct': 0.005 # 0.5% tolerance of target_qty
}

result = bex.compute_execution_delta(order_request)
display(result)

bex.submit_spot_order(
    symbol=result['symbol'], 
    side=result['side'],
    quantity=result['quantity'],
    order_type='LIMIT',
    price=2500,
    time_in_force='GTC',
    cancel_existing=True
)
#bex.cancel_existing_orders('ETHUSDT', 'SPOT')

Auto-calculated limit price at mid (if-available): 2442.3


{'symbol': 'ETHUSDT', 'side': 'BUY', 'quantity': 0.0819, 'price': 2442.3}

Reject Spot BUY Limit: price 2500 crosses best ask 2442.52


In [853]:
order_request = {
    'symbol': 'ETHUSDT', 
    'target_quantity': None,
    'target_notional': 200,
    'market_type': 'FUT', 
    'tol_pct': 0.005 # 0.5% tolerance of target_qty
}

result = bex.compute_execution_delta(order_request)
display(result)

bex.submit_fut_order(
    symbol=result['symbol'], 
    side=result['side'],
    quantity=result['quantity'],
    order_type='LIMIT',
    price=2500,
    time_in_force='GTC',
    cancel_existing=True,
    position_side='LONG' # hedge-mode
)
#bex.cancel_existing_orders('ETHUSDT', 'FUT')

Auto-calculated limit price at mid (if-available): 2441.86


{'symbol': 'ETHUSDT', 'side': 'BUY', 'quantity': 0.082, 'price': 2441.86}

Reject Futures BUY Limit: price 2500 crosses best ask 2441.87
